In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab')


In [ ]:
os.chdir( "./net") 
!ls


In [ ]:
filepath_list = []
file_dir = "./savedata"
for  file_name in  os.listdir(  file_dir) :
    temppath  =   os.path.join(  os.path.abspath(file_dir)  ,file_name   ) 
    filepath_list.append(temppath )


In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import  os 

In [ ]:
N = 10
import pandas  as pd
data = pd.DataFrame()
for path in filepath_list[:N]:
    df1 = pd.read_csv(path, header=None)
    df1.columns = ['iid', 'uid', 'rating' ,'date']
    df1= df1.loc[:,['iid', 'uid', 'rating']]
    data = pd.concat([data, df1] , axis  = 0 )
    
print("开始读入数据....")
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data, reader=reader)
train_set = data.build_full_trainset()
print("数据加载成功....")


In [ ]:
bsl_options = {'method': 'sgd', 'n_epochs': 5}
print("als 算法启动....")

algo = BaselineOnly(bsl_options=bsl_options)


In [ ]:
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

In [ ]:
basedir  ="./probegood"
newfilepath_list =[] 
for  file_name in  os.listdir(  basedir) :
    temppath  =   os.path.join(  os.path.abspath(basedir)  ,file_name   ) 
    newfilepath_list.append(temppath )
    
#排序    
sorted( newfilepath_list) 
    
# NN 控制测试集中文件的格式
NN = 10  
testsets  = pd.DataFrame()
for j in  range(NN):
    probe_csv_path =newfilepath_list[j]
    probedata = pd.read_table(  probe_csv_path ,delimiter= "," ,skiprows=1 , header =None )
    probedata.columns = ["user","item","rating","timestamp"]
    probedata  = probedata.loc[: ,["user","item","rating"] ]
    testsets = pd.concat(   [testsets   , probedata ]  ,axis  =0  )


In [ ]:
reader = Reader(line_format='user item rating', sep=',')
probedata= Dataset.load_from_df(testsets, reader=reader)
predictions = algo.test(testsets.values)
accuracy.rmse(predictions, verbose=True)


#### 文件一共大约500M数据 